In [29]:
# Dependencies and Setup
import gmaps
import pandas as pd
import requests
import json
import numpy as np

# Google developer API key
from api_config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [2]:
# Read clean and bounded ride data File and store into Pandas data frame
ride_data = pd.read_csv("Data/Rides_Data_Clean_Bounded.csv")

# Read CrimeFile and store into Pandas data frame
crime_data = pd.read_csv("Data/Crime_Reports_Clean.csv")

In [3]:
# Use google places API to find top 60 bar locations
# Target city (Austin lat long)
target_city = "30.267153,-97.7430608"
typeplace="bar"

# Build the endpoint URL
target_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=%s&radius=10000&sensor=true&type=%s&key=%s"%(target_city,typeplace,g_key)

In [4]:
# request google places data
place_data = requests.get(target_url).json()

In [5]:
# populate variables with location data of Austin bars from placces API
lat=[]
long=[]

for results in place_data["results"]:
    lat.append(results["geometry"]["location"]["lat"])
    long.append(results["geometry"]["location"]["lng"])

# google places API returns 20 per request with a next page token if more results are available
# expectation is that 3 pages (60 locations) will be available.

next_page=place_data["next_page_token"]

In [6]:
# request next 20 bars from google places with next page token
target_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken=%s&key=%s"%(next_page,g_key)
place_data = requests.get(target_url).json()

In [7]:
# add next 20 bars to variables
for results in place_data["results"]:
    lat.append(results["geometry"]["location"]["lat"])
    long.append(results["geometry"]["location"]["lng"])
    
next_page=place_data["next_page_token"]

In [8]:
# request next 20 bars from google places with next page token
target_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken=%s&key=%s"%(next_page,g_key)
place_data = requests.get(target_url).json()

In [9]:
# add next 20 bars to variables
for results in place_data["results"]:
    lat.append(results["geometry"]["location"]["lat"])
    long.append(results["geometry"]["location"]["lng"])

In [10]:
# make datafram of bar locations
bar_locations_df = pd.DataFrame()
bar_locations_df['lat']  = lat
bar_locations_df['long']  = long

In [11]:
# check number of bar locations collected
bar_locations_df.count()

lat     60
long    60
dtype: int64

In [12]:
# data samples are too large to create useful charts so we will take a random sample of 1500 entries from each
crime_locations = crime_data[['Latitude','Longitude']].sample(n=1500, random_state=1 )
crime_locations.head()

,Latitude,Longitude
5169,30.270560,-97.736735
3237,30.200441,-97.764447
6203,30.218296,-97.755799
1743,30.346646,-97.696573
1321,30.326516,-97.726568


In [13]:
# data samples are too large to create useful charts so we will take a random sample of 1500 entries from each
locations = ride_data[["start_location_lat", "start_location_long"]].sample(n=1500, random_state=1 )

# drop locations collected but the data is not currently useful for our charts
drop_locations = ride_data[["end_location_lat", "end_location_long"]].sample(n=1500, random_state=1 )

In [14]:
# check random sample counts
print(locations.count())
print(crime_locations.count())

start_location_lat     1500
start_location_long    1500
dtype: int64
Latitude     1500
Longitude    1500
dtype: int64


In [32]:
# Crime locatijons by point with heatmap of rideshare pickups
fig = gmaps.figure(zoom_level=13, center=(30.2672,-97.7431))

# symbol layer of crime locations
symbol_layer = gmaps.symbol_layer(crime_locations, fill_color='white', stroke_color='blue', scale=2, stroke_opacity=.25)

# heat layer of rideshare pickup
heat_layer = gmaps.heatmap_layer(locations, dissipating=True, max_intensity=1,point_radius=12)

# Add symbol layer
fig.add_layer(symbol_layer)

# Add heatmap layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [16]:
# Rideshare locatijons by point with heatmap of crime locations
fig = gmaps.figure(zoom_level=13, center=(30.2672,-97.7431))

# Create symbol layer ride share locations
symbol_layer = gmaps.symbol_layer(locations, fill_color='white', stroke_color='blue', scale=2, stroke_opacity=.25)

# Create heat layer of crime locations
heat_layer = gmaps.heatmap_layer(crime_locations, dissipating=True, max_intensity=1,point_radius=12)

# add symbol layer
fig.add_layer(symbol_layer)

# add hgeatmap
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Rideshare dropoff locations by point with heatmap of crime pickups (not used in presentation)
fig = gmaps.figure(zoom_level=13, center=(30.2672,-97.7431))

# ride share drop off symbol layer
symbol_layer = gmaps.symbol_layer(drop_locations, fill_color='white', stroke_color='green', scale=2, stroke_opacity=.25)

# Create heat layer
heat_layer = gmaps.heatmap_layer(crime_locations, dissipating=True, max_intensity=1,point_radius=12)

# Create symbol layer
fig.add_layer(symbol_layer)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Plot bar locations against crime heat map
fig = gmaps.figure(zoom_level=13, center=(30.2672,-97.7431))

# symbol layer of bar locations
symbol_layer = gmaps.symbol_layer(bar_locations_df, fill_color='white', stroke_color='blue', scale=4, stroke_opacity=.25)

# Create heat layer of crime
heat_layer = gmaps.heatmap_layer(crime_locations, dissipating=True, max_intensity=1,point_radius=12)

# add bar locations
fig.add_layer(symbol_layer)

# Add crime heatmap
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Plot bar locations against rideshare heat map
fig = gmaps.figure(zoom_level=13, center=(30.2672,-97.7431))

# symbol layer of bar locations
symbol_layer = gmaps.symbol_layer(bar_locations_df, fill_color='white', stroke_color='blue', scale=4, stroke_opacity=.25)

# Create heat layer of ride share
heat_layer = gmaps.heatmap_layer(locations, dissipating=True, max_intensity=1,point_radius=12)

#fig.add_layer(markers)
fig.add_layer(symbol_layer)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))